In [1]:
import io
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from PIL import Image
from ultralytics import YOLO
import pandas as pd

In [2]:
# Initialize S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [3]:
UMBRA_BUCKET = 'umbra-open-data-catalog'
prefix = 'sar-data/tasks/ship_detection_testdata/'

In [4]:
response = s3.list_objects_v2(Bucket=UMBRA_BUCKET, Prefix=prefix, Delimiter='/')

In [5]:
response['CommonPrefixes'][:20]

[{'Prefix': 'sar-data/tasks/ship_detection_testdata/00174818-00fe-4c89-bf45-cc6149253707/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/00dee081-76ee-413a-8af2-e6be1c6ab8d0/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/00f0e649-8618-45cc-9c6d-629a1576ec54/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/011be8cb-1867-44cb-80ba-706d1e94f94e/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/0121e119-91e5-41b4-b428-b5b8825adae7/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/012fe132-d975-4d27-99f6-c70fd7fd6471/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/015e671e-1893-4847-a1e8-3b774964ce9f/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/018a5725-421c-4089-bea7-625f904dfd07/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/018f3ac1-eff3-487c-95fd-d869db502c57/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/01d0cc4e-80fb-4c32-8eb4-3d9f6cc8fe1b/'},
 {'Prefix': 'sar-data/tasks/ship_detection_testdata/01e48f3a-d9b4-4fe7

In [6]:
files = s3.list_objects_v2(Bucket=UMBRA_BUCKET, Prefix='sar-data/tasks/ship_detection_testdata/')

In [7]:
tif_files = [ f for f in files['Contents'] if ('.tif' in f['Key']) ]

In [8]:
len(tif_files)

217

In [9]:
def load_image_from_s3(bucket_name, object_key):
    """
    Load an image directly from an S3 bucket.
    
    Args:
        bucket_name (str): Name of the S3 bucket.
        object_key (str): Path to the image in the bucket.

    Returns:
        PIL.Image: The loaded image.
    """
    # Get the object from S3
    s3_response = s3.get_object(Bucket=bucket_name, Key=object_key)
    
    # Read the image data and load it as a PIL image
    image_data = s3_response['Body'].read()
    image = Image.open(io.BytesIO(image_data)).convert("RGB")
    return image

In [10]:
def process_images_with_yolo(model, images):
    """
    Run YOLO model inference on a loaded image.

    Args:
        model (YOLO): YOLO model instance.
        image (PIL.Image): Loaded image to process.

    Returns:
        results: YOLO model inference results.
    """
    # Convert PIL image to a format compatible with YOLO (typically numpy or tensor)
    # helpful documentation: https://docs.ultralytics.com/reference/engine/model/
    results = model(images, save=True, save_txt=True)
    return results

In [11]:
# Load YOLO model
model = YOLO('yolo11n-obb.pt')

In [12]:
# because satellite images are large
Image.MAX_IMAGE_PIXELS = None

In [13]:
files

{'ResponseMetadata': {'RequestId': 'RWNXSYHH5TP9GF6M',
  'HostId': 'mBrCKKvNdcU67OYFZh9uBGt/pO8EhprwniBpJiZEidZhb1zoQtYILlUATm82KKRYIVkgtSK1XnI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mBrCKKvNdcU67OYFZh9uBGt/pO8EhprwniBpJiZEidZhb1zoQtYILlUATm82KKRYIVkgtSK1XnI=',
   'x-amz-request-id': 'RWNXSYHH5TP9GF6M',
   'date': 'Wed, 20 Nov 2024 22:26:22 GMT',
   'x-amz-bucket-region': 'us-west-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': True,
 'Contents': [{'Key': 'sar-data/tasks/ship_detection_testdata/00174818-00fe-4c89-bf45-cc6149253707/2023-04-15-01-17-53_UMBRA-05/2023-04-15-01-17-53_UMBRA-05_CPHD.cphd',
   'LastModified': datetime.datetime(2024, 5, 30, 1, 46, 47, tzinfo=tzutc()),
   'ETag': '"4a8f7f93ea50d0229028ed4f83f5d117-704"',
   'Size': 5897272528,
   'StorageClass': 'INTELLIGENT_TIERING'},
  {'Key': 'sar-data/tasks/ship_detection_testdata/00174818-00fe-4c89-bf45-cc6149

In [14]:
# not a safe way to call the object_key, but we can generalize it later and this takes a long time because images are huge
images = [ load_image_from_s3(UMBRA_BUCKET, f['Key']) for f in tif_files[:2] ]

In [15]:
yolo_results = process_images_with_yolo(model, images)


0: 1024x1024 168.2ms
1: 1024x1024 (no detections), 168.2ms
Speed: 13.5ms preprocess, 168.2ms inference, 1.3ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/obb/predict8
1 label saved to runs/obb/predict8/labels


In [17]:
yolo_results[0]

ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'plane', 1: 'ship', 2: 'storage tank', 3: 'baseball diamond', 4: 'tennis court', 5: 'basketball court', 6: 'ground track field', 7: 'harbor', 8: 'bridge', 9: 'large vehicle', 10: 'small vehicle', 11: 'helicopter', 12: 'roundabout', 13: 'soccer ball field', 14: 'swimming pool'}
obb: ultralytics.engine.results.OBB object
orig_img: array([[[19, 19, 19],
        [ 7,  7,  7],
        [17, 17, 17],
        ...,
        [72, 72, 72],
        [61, 61, 61],
        [25, 25, 25]],

       [[61, 61, 61],
        [37, 37, 37],
        [ 0,  0,  0],
        ...,
        [27, 27, 27],
        [55, 55, 55],
        [61, 61, 61]],

       [[60, 60, 60],
        [35, 35, 35],
        [ 0,  0,  0],
        ...,
        [54, 54, 54],
        [46, 46, 46],
        [56, 56, 56]],

       ...,

       [[14, 14, 14],
        [40, 40, 40],
        [28, 28, 28],
        ...,
        [51, 51, 51],
   

In [18]:
# Documentation on results file: https://docs.ultralytics.com/datasets/obb/#supported-obb-dataset-formats
def load_results_to_add_metadata(i, results, s3_key):
    cols = ['class_index', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']
    try:
        run_data = pd.read_csv(results.save_dir + '/labels/image' + str(i) + '.txt', sep=' ', names=cols)
        run_data['s3_object'] = s3_key
        run_data['class_name'] = run_data['class_index'].apply(lambda x: results[0].names[x])
        return run_data
    except FileNotFoundError:
        pass

In [19]:
# Example of the end result
df = pd.concat(
    [
        load_results_to_add_metadata(
            images.index(i),
            yolo_results[images.index(i)],
            tif_files[images.index(i)]['Key']
        )
        for i in images 
    ]
)

In [20]:
df

,class_index,x1,y1,x2,y2,x3,y3,x4,y4,s3_object,class_name
0,1,0.641826,0.504921,0.648928,0.514356,0.680818,0.490352,0.673716,0.480917,sar-data/tasks/ship_detection_testdata/0017481...,ship
1,1,0.494022,0.159517,0.504100,0.159834,0.504873,0.135249,0.494794,0.134933,sar-data/tasks/ship_detection_testdata/0017481...,ship
2,1,0.483884,0.166160,0.492525,0.166233,0.492762,0.138505,0.484121,0.138432,sar-data/tasks/ship_detection_testdata/0017481...,ship
3,1,0.682541,0.472303,0.687806,0.478396,0.707215,0.461626,0.701950,0.455533,sar-data/tasks/ship_detection_testdata/0017481...,ship
4,1,0.213718,0.423543,0.213812,0.412943,0.175133,0.412603,0.175040,0.423203,sar-data/tasks/ship_detection_testdata/0017481...,ship
5,1,0.156884,0.736837,0.157210,0.751664,0.198280,0.750763,0.197955,0.735936,sar-data/tasks/ship_detection_testdata/0017481...,ship


In [45]:
len(df)

14

---

---

---